<a href="https://www.kaggle.com/code/kadalimounikamehar/house-xgbpredict?scriptVersionId=128600251" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import xgboost
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
main_df=df.copy()

In [4]:
main_df.shape

(1460, 81)

In [5]:
tf=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
tf.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [6]:
pd.set_option('display.max_rows',None)

In [7]:
df.isnull().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [8]:
df.drop(['Alley'],axis=1,inplace=True)

In [9]:
df.drop(['PoolQC','Fence','MiscFeature'],axis=1,inplace=True)

In [10]:
df.shape

(1460, 77)

In [11]:
tf.isnull().sum()

Id                  0
MSSubClass          0
MSZoning            4
LotFrontage       227
LotArea             0
Street              0
Alley            1352
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType         16
MasVnrArea         15
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           44
BsmtCond           45
BsmtExposure       44
BsmtFinType1       42
BsmtFinSF1          1
BsmtFinType2       42
BsmtFinSF2          1
BsmtUnfSF           1
TotalBsmtSF         1
Heating             0
HeatingQC           0
CentralAir          0
Electrical          0
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [12]:
tf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [13]:
df['LotFrontage']=df['LotFrontage'].fillna(df['LotFrontage'].mean())

In [14]:
df['BsmtQual'].value_counts()

TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64

In [15]:
df['BsmtQual']=df['BsmtQual'].fillna(df['BsmtQual'].mode()[0])

In [16]:
df['BsmtCond']=df['BsmtCond'].fillna(df['BsmtCond'].mode()[0])

In [17]:
df['BsmtExposure']=df['BsmtExposure'].fillna(df['BsmtExposure'].mode()[0])
df['BsmtFinType1']=df['BsmtFinType1'].fillna(df['BsmtFinType1'].mode()[0])
df['BsmtFinType2']=df['BsmtFinType2'].fillna(df['BsmtFinType2'].mode()[0])
df['FireplaceQu']=df['FireplaceQu'].fillna(df['FireplaceQu'].mode()[0])
df['GarageType']=df['GarageType'].fillna(df['GarageType'].mode()[0])
df['GarageYrBlt']=df['GarageYrBlt'].fillna(df['GarageYrBlt'].mean())
df['GarageFinish']=df['GarageFinish'].fillna(df['GarageFinish'].mode()[0])
df['GarageQual']=df['GarageQual'].fillna(df['GarageQual'].mode()[0])
df['GarageCond']=df['GarageCond'].fillna(df['GarageCond'].mode()[0])

In [18]:
df['Electrical']=df['Electrical'].fillna(df['Electrical'].mode()[0])

In [19]:
df['MasVnrType']=df['MasVnrType'].fillna(df['MasVnrType'].mode()[0])
df['MasVnrArea']=df['MasVnrArea'].fillna(df['MasVnrArea'].mean())

In [20]:
tf.drop(['Alley','PoolQC','Fence','MiscFeature'],axis=1,inplace=True)

In [21]:
tf['BsmtQual']=tf['BsmtQual'].fillna(tf['BsmtQual'].mode()[0])
tf['BsmtCond']=tf['BsmtCond'].fillna(tf['BsmtCond'].mode()[0])
tf['BsmtExposure']=tf['BsmtExposure'].fillna(tf['BsmtExposure'].mode()[0])
tf['BsmtFinType1']=tf['BsmtFinType1'].fillna(tf['BsmtFinType1'].mode()[0])
tf['BsmtFinType2']=tf['BsmtFinType2'].fillna(tf['BsmtFinType2'].mode()[0])
tf['BsmtFinSF1']=tf['BsmtFinSF1'].fillna(tf['BsmtFinSF1'].mean())
tf['FireplaceQu']=tf['FireplaceQu'].fillna(tf['FireplaceQu'].mode()[0])
tf['GarageType']=tf['GarageType'].fillna(tf['GarageType'].mode()[0])
tf['GarageYrBlt']=tf['GarageYrBlt'].fillna(tf['GarageYrBlt'].mean())
tf['GarageFinish']=tf['GarageFinish'].fillna(tf['GarageFinish'].mode()[0])
tf['GarageQual']=tf['GarageQual'].fillna(tf['GarageQual'].mode()[0])
tf['GarageCond']=tf['GarageCond'].fillna(tf['GarageCond'].mode()[0])

In [22]:
tf['BsmtFinSF2']=tf['BsmtFinSF2'].fillna(tf['BsmtFinSF2'].mean())
tf['BsmtUnfSF']=tf['BsmtUnfSF'].fillna(tf['BsmtUnfSF'].mean())
tf['TotalBsmtSF']=tf['TotalBsmtSF'].fillna(tf['TotalBsmtSF'].mean())
tf['LotFrontage']=tf['LotFrontage'].fillna(tf['LotFrontage'].mean())
tf['Utilities']=tf['Utilities'].fillna(tf['Utilities'].mode()[0])
tf['Exterior1st']=tf['Exterior1st'].fillna(tf['Exterior1st'].mode()[0])
tf['Exterior2nd']=tf['Exterior2nd'].fillna(tf['Exterior2nd'].mode()[0])
tf['MasVnrType']=tf['MasVnrType'].fillna(tf['MasVnrType'].mode()[0])
tf['BsmtFullBath']=tf['BsmtFullBath'].fillna(tf['BsmtFullBath'].mean())
tf['BsmtHalfBath']=tf['BsmtHalfBath'].fillna(tf['BsmtHalfBath'].mean())

In [23]:
columns=['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType',
         'HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual'
         ,'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','FireplaceQu',
        'GarageType','GarageQual','GarageFinish','GarageCond','PavedDrive','SaleType','SaleCondition']

In [24]:
len(columns)

39

In [25]:
df.shape

(1460, 77)

In [26]:
tf.shape

(1459, 76)

In [27]:
c = pd.concat([df,tf],axis=0)
c.shape

(2919, 77)

In [28]:
final_df = pd.concat([df,tf],axis=0)
final_df.shape

(2919, 77)

In [29]:
def one_hot_encode(cols):
    df_final=final_df
    i=0
    for x in cols:
        print(x)
        df1 = pd.get_dummies(final_df[x],drop_first=True)  
        final_df.drop([x],axis=1,inplace=True)
        if i==0:
            df_final=df1.copy()
        else:
            df_final=pd.concat([df_final,df1],axis=1)
        i=i+1
    df_final = pd.concat([df_final,final_df],axis=1)
    return df_final

In [30]:
fd = one_hot_encode(columns)

MSZoning
Street
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageQual
GarageFinish
GarageCond
PavedDrive
SaleType
SaleCondition


In [31]:
fd=fd.loc[:,~fd.columns.duplicated()]

In [32]:
fd.shape

(2919, 178)

In [33]:
fd.head()

,FV,RH,RL,RM,Pave,IR2,IR3,Reg,HLS,Low,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0,0,1,0,1,0,0,1,0,0,...,0,61,0,0,0,0,0,2,2008,208500.0
1,0,0,1,0,1,0,0,1,0,0,...,298,0,0,0,0,0,0,5,2007,181500.0
2,0,0,1,0,1,0,0,0,0,0,...,0,42,0,0,0,0,0,9,2008,223500.0
3,0,0,1,0,1,0,0,0,0,0,...,0,35,272,0,0,0,0,2,2006,140000.0
4,0,0,1,0,1,0,0,0,0,0,...,192,84,0,0,0,0,0,12,2008,250000.0


In [34]:
fd_Train=fd.iloc[:1460,:]
fd_Test=fd.iloc[1460:,:]

In [35]:
fd_Train.head()

,FV,RH,RL,RM,Pave,IR2,IR3,Reg,HLS,Low,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0,0,1,0,1,0,0,1,0,0,...,0,61,0,0,0,0,0,2,2008,208500.0
1,0,0,1,0,1,0,0,1,0,0,...,298,0,0,0,0,0,0,5,2007,181500.0
2,0,0,1,0,1,0,0,0,0,0,...,0,42,0,0,0,0,0,9,2008,223500.0
3,0,0,1,0,1,0,0,0,0,0,...,0,35,272,0,0,0,0,2,2006,140000.0
4,0,0,1,0,1,0,0,0,0,0,...,192,84,0,0,0,0,0,12,2008,250000.0


In [36]:
fd_Test.head()

,FV,RH,RL,RM,Pave,IR2,IR3,Reg,HLS,Low,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0,1,0,0,1,0,0,1,0,0,...,140,0,0,0,120,0,0,6,2010,NaN
1,0,0,1,0,1,0,0,0,0,0,...,393,36,0,0,0,0,12500,6,2010,NaN
2,0,0,1,0,1,0,0,0,0,0,...,212,34,0,0,0,0,0,3,2010,NaN
3,0,0,1,0,1,0,0,0,0,0,...,360,36,0,0,0,0,0,6,2010,NaN
4,0,0,1,0,1,0,0,0,1,0,...,0,82,0,0,144,0,0,1,2010,NaN


In [37]:
fd_Test.drop(['SalePrice'],axis=1,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
X_Train=fd_Train.drop(['SalePrice'],axis=1)
Y_Train=fd_Train['SalePrice']

In [39]:
Y_Train.shape

(1460,)

In [40]:
r=xgboost.XGBRegressor()

In [41]:
r.fit(X_Train,Y_Train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [42]:
y_pred=r.predict(fd_Test)
y_pred

array([124159.03, 151698.84, 187626.66, ..., 169108.94, 104867.45,
       222425.06], dtype=float32)

In [43]:
y_pred.shape

(1459,)

In [44]:
df.shape

(1460, 77)

In [45]:
main_df.shape

(1460, 81)

In [46]:
fd_Test.shape

(1459, 177)

In [47]:
output = pd.DataFrame({'Id':fd_Test.Id, 'SalePrice': y_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
